In [7]:
%%writefile mapper1.py

import sys
import re

for line in sys.stdin:
    try:
        article_id, content = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", content,flags=re.UNICODE)
    for word in words:
        sys.stderr.write(f'reporter:counter:Wiki stats,Total words,1\n')
        print(f'{word.lower()}\t1')

Overwriting mapper1.py


In [2]:
%%writefile reducer1.py

import sys

current_key = None
total_amt = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print(f'{current_key}\t{total_amt}')
        total_amt = 0
        current_key = key
    total_amt += count

if current_key:
    print(f'{current_key}\t{total_amt}')

Overwriting reducer1.py


In [3]:
%%writefile mapper_sort.py

import sys

for line in sys.stdin:
    try:
        word, amt = line.strip().split('\t',1)
        amt = int(amt)
    except ValueError as e:
        continue
    
    print(f'{amt}\t{word}')

Overwriting mapper_sort.py


In [4]:
%%writefile reducer_sort.py

import sys

for line in sys.stdin:
    try:
        amt, word = line.strip().split('\t',1)
        amt = int(amt)
    except ValueError as e:
        continue
    
    print(f'{word}\t{amt}')

Overwriting reducer_sort.py


In [127]:
%%bash

OUT_DIR="assignment1_"$(date +"%s%6N")
HADOOP_JAR="/opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar"

hdfs dfs -rm -r -skipTrash assignment* > /dev/null

# Code for your first job
yarn jar ${HADOOP_JAR} \
    -D mapreduce.job.name="First Job" \
    -D mapreduce.job.reduces=4 \
    -files mapper1.py,reducer1.py \
    -mapper "python3 mapper1.py" \
    -combiner "python3 reducer1.py" \
    -reducer "python3 reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

OUT_DIR2="final_result"

hdfs dfs -rm -r -skipTrash final_result* > /dev/null

# Code for your second job
yarn jar ${HADOOP_JAR} \
    -D mapreduce.job.name="Second Job" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.map.output.key.field.separator=\t \
    -D mapreduce.partition.keycomparator.options="-k1,1nr" \
    -files mapper_sort.py,reducer_sort.py \
    -mapper "python3 mapper_sort.py" \
    -reducer "python3 reducer_sort.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null



hdfs dfs -cat ${OUT_DIR2}/part-00000 | sed -n '7p'


the	831724
of	448016
and	344105
in	298796
to	242846
a	242158
is	126579


19/03/18 13:53:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/18 13:53:43 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/18 13:53:44 INFO mapred.FileInputFormat: Total input files to process : 1
19/03/18 13:53:45 INFO mapreduce.JobSubmitter: number of splits:2
19/03/18 13:53:45 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1552910315332_0017
19/03/18 13:53:45 INFO impl.YarnClientImpl: Submitted application application_1552910315332_0017
19/03/18 13:53:45 INFO mapreduce.Job: The url to track the job: http://12da6f52529f:8088/proxy/application_1552910315332_0017/
19/03/18 13:53:45 INFO mapreduce.Job: Running job: job_1552910315332_0017
19/03/18 13:53:51 INFO mapreduce.Job: Job job_1552910315332_0017 running in uber mode : false
19/03/18 13:53:51 INFO mapreduce.Job:  map 0% reduce 0%
19/03/18 13:54:07 INFO mapreduce.Job:  map 21% reduce 0%
19/03/18 13:54:13 INFO mapreduce.Job:  map 32% reduce 0%
19/03/18 13:54:19 INFO 

CPU times: user 10 ms, sys: 10 ms, total: 20 ms
Wall time: 1min 44s
